- from one to three scaling laws
    - pre-training
    - post-training
        - RL
    - test-time scaling
        - 根据 query，实时对计算弹性伸缩；

### Point-wise GRM

- principle => critic => score
    - 奖励过程思维链化（CoT）了
    - 定标准，写评价，再打分；